In [ ]:
pip install implicit
pip install lightfm

     |████████████████████████████████| 1.1MB 6.1MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419493 sha256=6e4213e8d57525ae0ab8040032886e2f2e2443b1cd5019a53bc11010664fe34c
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -R  /content/gdrive/MyDrive/Recomendation/srcrec  /

In [ ]:
path_to_src = "/content/gdrive/MyDrive/Recomendation/"

In [ ]:
if path_to_src not in sys.path:
    sys.path.append(path_to_src)

# Написанные функции
from srcrec.metrics import precision_at_k, recall_at_k
from srcrec.utils import prefilter_items
from srcrec.recommenders import MainRecommender

In [ ]:
path_to_files = "/content/gdrive/MyDrive/Recomendation"
data          = pd.read_csv(os.path.join(path_to_files, "df_data.csv"))
user_features = pd.read_csv(os.path.join(path_to_files, "user_features.csv"))
item_features = pd.read_csv(os.path.join(path_to_files, "item_features.csv"))

In [ ]:
# схема обучения и валидации
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)

val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,999999,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,999999,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
recommender = MainRecommender(data_train_lvl_1, item_features, user_features)

user_item_matrix


GPU training requires factor size to be a multiple of 32. Increasing factors from 50 to 64.


user_feat
item_feat
ALS model


ALS TF-IDF model


ALS BM25 model
LightFM model
Own


In [ ]:
users_lvl_2_pred_als = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_lvl_2_pred_als.columns = ['user_id']

In [ ]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

In [ ]:
k_n = 5
num = 40

In [ ]:
popularity = data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
item_list = popularity.sort_values('n_sold', ascending=False).head(num+1).item_id.tolist()
item_list = item_list[1:num+1]
len(item_list)

40

In [ ]:
user_train_list = data_train_lvl_1["user_id"].unique().tolist()

In [ ]:
def get_pred_als(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [ ]:
def get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_tfidf_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [ ]:
def get_pred_als_bm25(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_bm25_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [ ]:
def get_pred_own(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_own_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [ ]:
def get_pred_similar_items(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_similar_items_recommendation(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [ ]:
users_lvl_2_pred_als['predictions'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec"].mean()

0.21958863858961494

In [ ]:
users_lvl_2_pred_als["rec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec"].mean()

0.03980328273375497

In [ ]:
users_lvl_2_pred_als['predictions_tfidf'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_tfidf"].mean()

0.16287952987267165

In [ ]:
users_lvl_2_pred_als["rec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_tfidf"].mean()

0.030182187673331817

In [ ]:
users_lvl_2_pred_als['predictions_bm25'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_bm25(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_bm25"].mean()

0.14417238001958693

In [ ]:
users_lvl_2_pred_als["rec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_bm25"].mean()

0.026093413904243197

In [ ]:
users_lvl_2_pred_als.head()

,user_id,predictions,prec,rec,predictions_tfidf,prec_tfidf,rec_tfidf,predictions_bm25,prec_bm25,rec_bm25
0,338,"[845208, 981760, 904360, 1037863, 1026118, 108...",0.4,0.026667,"[923746, 845208, 904360, 1004906, 1026118, 103...",0.4,0.026667,"[819840, 1025650, 1084036, 848071, 845208, 871...",0.0,0.000000
1,2120,"[1082185, 1106523, 981760, 1029743, 995242, 11...",0.0,0.000000,"[1082185, 1029743, 995242, 981760, 1127831, 11...",0.0,0.000000,"[1082185, 849843, 981760, 995055, 1110843, 986...",0.0,0.000000
2,2324,"[995242, 1029743, 1082185, 5978656, 916122, 98...",0.6,0.071429,"[5569230, 5978656, 5569471, 1136257, 1039156, ...",0.2,0.023810,"[1048068, 967760, 1082185, 6979128, 907392, 96...",0.6,0.071429
3,514,"[878996, 866211, 1005186, 1024306, 1082185, 11...",0.8,0.125000,"[1062002, 1023720, 1022254, 1024306, 866211, 1...",0.6,0.093750,"[1082185, 1126786, 835595, 923169, 6391534, 87...",0.6,0.093750
4,1762,"[1136257, 1003188, 986912, 1068719, 1084331, 1...",0.2,0.011765,"[1110572, 1084331, 1096036, 1003188, 8019716, ...",0.2,0.011765,"[866211, 868971, 1029743, 876932, 1071939, 100...",0.2,0.011765


### Строим классификатор

In [ ]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: get_pred_als(recommender, user_train_list, item_list, x, num=num))

In [ ]:
users_lvl_2.head()

,user_id,candidates
0,2070,"[5569471, 1056509, 1107553, 1029743, 833025, 9..."
1,2021,"[951590, 1082185, 1044078, 981760, 826249, 109..."
2,1753,"[1106523, 1005186, 1085604, 1110572, 986912, 8..."
3,2120,"[1082185, 1106523, 981760, 1029743, 995242, 11..."
4,1346,"[1082185, 1127831, 866211, 1058997, 1126899, 8..."


In [ ]:
users_lvl_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     2154 non-null   int64 
 1   candidates  2154 non-null   object
dtypes: int64(1), object(1)
memory usage: 33.8+ KB


In [ ]:
df=pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values)})

In [ ]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [ ]:
targets_lvl_2.shape

(94268, 3)

In [ ]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left') #
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left') #

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,bill_avg,week_last_bill,items_per_week
0,2070,5569471,0.0,1208,GROCERY,National,SOFT DRINKS,SOFT DRINKS 12/18&15PK CAN CAR,12 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,95.0,29.814815
1,2070,1056509,0.0,69,GROCERY,Private,MILK BY-PRODUCTS,COTTAGE CHEESE,24 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,95.0,29.814815


In [ ]:
targets_lvl_2.shape

(94268, 19)

In [ ]:
targets_lvl_2.fillna(0, inplace=True)

In [ ]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [ ]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'bill_avg',
 'week_last_bill',
 'items_per_week']

In [ ]:
lgb = LGBMClassifier(objective='binary', 
                     n_estimators=1500,
                     learning_rate=0.008,
                     max_depth=7, 
                     verbose=1,
                     categorical_column=cat_feats)

hist = lgb.fit(X_train, y_train, 
               eval_set=[(X_train, y_train)],
               eval_metric=['logloss'],
               early_stopping_rounds=50,
               verbose=0)

train_preds = lgb.predict_proba(X_train)

In [ ]:
hist.evals_result_['training']['binary_logloss'][-10:]

[0.3783245303369548,
 0.37829173796905213,
 0.37827316826989055,
 0.37822250670575436,
 0.3782002306062754,
 0.378185395897456,
 0.3781676667100933,
 0.37815177105480346,
 0.3781320112918844,
 0.378117263478605]

In [ ]:
classifer_prediction = X_train[['user_id', 'item_id']]
classifer_prediction["pred"] = train_preds[:, 1]
classifer_prediction.tail()

,user_id,item_id,pred
94263,1745,947798,0.035994
94264,1745,965766,0.050346
94265,1745,1033142,0.010672
94266,1745,878996,0.009463
94267,1745,1043590,0.018260


In [ ]:
res_val = classifer_prediction.loc[classifer_prediction["pred"]>0.5, ["user_id", "item_id"]]
res_val.shape

(9791, 2)

In [ ]:
pred_lvl_2 = res_val.groupby('user_id')['item_id'].unique().reset_index()
pred_lvl_2.columns=['user_id', 'pred']
pred_lvl_2.head()

,user_id,pred
0,1,"[1029743, 995242, 1082185, 940947, 5978656, 59..."
1,7,"[1126899, 1082185, 1106523]"
2,8,"[1082185, 1029743]"
3,13,"[1029743, 5978656, 1058997, 995242, 1082185, 9..."
4,17,"[1029743, 995242, 1082185]"


In [ ]:
result_lvl_2 = result_lvl_2.merge(pred_lvl_2, on='user_id', how='left')
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[1029743, 995242, 1082185, 940947, 5978656, 59..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...",NaN
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 11...",NaN
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1126899, 1082185, 1106523]"
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1082185, 1029743]"


In [ ]:
user_pred_nan = result_lvl_2.loc[result_lvl_2["pred"] != result_lvl_2["pred"], 'user_id'].values
len(user_pred_nan)

1323

In [ ]:
set_tr_1 = set(data_train_lvl_1.user_id.unique())
len(set_tr_1)

2498

In [ ]:
user_list = set(user_pred_nan).intersection(set_tr_1)
len(user_list)

1322

In [ ]:
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "pred"] =\
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_als_recommendations(x, rec_num=k_n))
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[1029743, 995242, 1082185, 940947, 5978656, 59..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...","[1092026, 1053690, 5569230, 1106523, 951590]"
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 11...","[878996, 866211, 1023720, 1024306, 923746]"
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1126899, 1082185, 1106523]"
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1082185, 1029743]"


In [ ]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,pred
1622,1984,"[999999, 847344, 859237, 1064380, 1074516, 122...",NaN


In [ ]:
popularity = recommender.prep_data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5 = popularity.sort_values('n_sold', ascending=False).head(6).item_id.tolist()[1:6]
top_5

[1082185, 995242, 1029743, 1133018, 981760]

In [ ]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"] =\
        result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "user_id"].apply(lambda x: top_5)

In [ ]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,pred
1622,1984,"[999999, 847344, 859237, 1064380, 1074516, 122...","[1082185, 995242, 1029743, 1133018, 981760]"


In [ ]:
len(result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"].values[0])

5

In [ ]:
res = []
res.extend(result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"].values[0])
res

[1082185, 995242, 1029743, 1133018, 981760]

In [ ]:
def fill_if_not_5(recommender, x):
    res = []
    res.extend(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0])
    pred_num = len(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0])
    if pred_num < 5:
        res.extend(recommender.get_als_recommendations(x, rec_num=5-pred_num))
    return res

In [ ]:
result_lvl_2["pred"] = result_lvl_2["user_id"].apply(lambda x: fill_if_not_5(recommender, x))

In [ ]:
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[1029743, 995242, 1082185, 940947, 5978656, 59..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...","[1092026, 1053690, 5569230, 1106523, 951590]"
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 11...","[878996, 866211, 1023720, 1024306, 923746]"
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1126899, 1082185, 1106523, 1126899, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1082185, 1029743, 5569230, 883404, 1005186]"


In [ ]:
result_lvl_2["rec"] = result_lvl_2['user_id'].apply(\
        lambda x: precision_at_k(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=5))

result_lvl_2['rec'].mean()

0.2554358472086164

### LightGBM Ranker

In [ ]:
X_train.shape

(94268, 18)

In [ ]:
X_train['num'] = np.arange(len(X_train))

In [ ]:
gr = X_train[['user_id', 'num']].groupby('user_id', sort=False).max()['num'].values

In [ ]:
grs = np.concatenate((np.array([gr[0]+1]), gr[1:] - gr[:-1]))

In [ ]:
X_train.shape, y_train.shape, np.sum(grs)

((94268, 19), (94268, 1), 94268)

In [ ]:
from lightgbm import LGBMRanker

lgb_params = { 
    'objective':'lambdarank',
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'learning_rate': 0.08,
    'max_depth': 5,
    'categorical_column': cat_feats,
    'random_state': 27,
    'verbose': 0,
    'is_unbalance': True
}

In [ ]:
lgb_rn = LGBMRanker(**lgb_params, silent=False)

eval_h = lgb_rn.fit(X_train, y_train, group=grs,
                    eval_set=[(X_train, y_train)], eval_group=[grs], 
                    eval_metric=['ndcg'],
                    eval_at=[5, 10 ], 
                    early_stopping_rounds=50, 
                    verbose=0)

In [ ]:
eval_h.evals_result_['training']['ndcg@5'][:3], eval_h.evals_result_['training']['ndcg@5'][-3:]

([0.5731919636025121, 0.5731919636025121, 0.5799634676327567],
 [0.7568685756025532, 0.7571228574297987, 0.7571228574297987])

In [ ]:
rank_preds = lgb_rn.predict(X_train)

In [ ]:
ranker_prediction = X_train[['user_id', 'item_id']]
ranker_prediction["pred"] = rank_preds
ranker_prediction.head()

,user_id,item_id,pred
0,2070,5569471,-1.016648
1,2070,1056509,-0.814604
2,2070,1107553,-0.252708
3,2070,1029743,-0.239741
4,2070,833025,-0.280658


In [ ]:
ranker_prediction = ranker_prediction.drop_duplicates()

In [ ]:
ranker_prediction.sort_values(by="pred", inplace=True, ascending=False)
ranker_prediction.head()

,user_id,item_id,pred
60990,2416,849697,1.982477
92293,2428,6533681,1.963839
2852,957,14019790,1.930329
68664,1139,856942,1.771721
51191,1541,849578,1.749958


In [ ]:
ranked_res = ranker_prediction.groupby('user_id')['item_id'].unique().reset_index()   
ranked_res

,user_id,item_id
0,1,"[856942, 1082185, 940947, 5577022, 1029743, 99..."
1,2,"[1082185, 1106523, 979707, 899624, 1053690, 96..."
2,4,"[883932, 1029743, 1082185, 1106523, 995242, 96..."
3,6,"[1082185, 1029743, 995242, 900802, 862349, 962..."
4,7,"[1126899, 1106523, 1082185, 1122358, 5591154, ..."
...,...,...
2149,2496,"[1082185, 1106523, 899624, 979707, 883404, 113..."
2150,2497,"[900802, 1029743, 1055646, 995242, 1082185, 87..."
2151,2498,"[1070820, 914190, 1053690, 1126899, 1082185, 1..."
2152,2499,"[5568378, 5569327, 1106523, 899624, 1082185, 1..."


In [ ]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2.merge(ranked_res, on='user_id', how='left')
result_lvl_2

,user_id,actual,item_id
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[856942, 1082185, 940947, 5577022, 1029743, 99..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...",NaN
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 11...","[1082185, 1029743, 995242, 900802, 862349, 962..."
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1126899, 1106523, 1082185, 1122358, 5591154, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1082185, 1044078, 1029743, 1070820, 1053690, ..."
...,...,...,...
2037,2496,[999999],"[1082185, 1106523, 899624, 979707, 883404, 113..."
2038,2497,"[1016709, 9835695, 999999, 845294, 871756, 873...","[900802, 1029743, 1055646, 995242, 1082185, 87..."
2039,2498,"[999999, 901776, 914190, 958382, 972437, 10398...","[1070820, 914190, 1053690, 1126899, 1082185, 1..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[5568378, 5569327, 1106523, 899624, 1082185, 1..."


In [ ]:
user_pred_nan = result_lvl_2.loc[result_lvl_2["item_id"] != result_lvl_2["item_id"], 'user_id'].values
len(user_pred_nan)

126

In [ ]:
set_tr_1 = set(data_train_lvl_1.user_id.unique())
len(set_tr_1)

2498

In [ ]:
user_list = set(user_pred_nan).intersection(set_tr_1)
len(user_list)

126

In [ ]:
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "item_id"] =\
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_als_recommendations(x, rec_num=k_n))
result_lvl_2.head()

,user_id,actual,item_id
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[856942, 1082185, 940947, 5577022, 1029743, 99..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...","[1092026, 1053690, 5569230, 1106523, 951590]"
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 11...","[1082185, 1029743, 995242, 900802, 862349, 962..."
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1126899, 1106523, 1082185, 1122358, 5591154, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1082185, 1044078, 1029743, 1070820, 1053690, ..."


In [ ]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,item_id
1622,1984,"[999999, 847344, 859237, 1064380, 1074516, 122...","[1029743, 1106523, 1082185, 1126899, 1070820, ..."


In [ ]:
result_lvl_2["rec"] = result_lvl_2['user_id'].apply(\
        lambda x: precision_at_k(result_lvl_2.loc[result_lvl_2["user_id"] == x, "item_id"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=5))

result_lvl_2['rec'].mean()

0.2906953966699305